In [ ]:
import time

def countdown_timer(seconds):
    while seconds > 0:
        mins, secs = divmod(seconds, 60)
        print(f"{mins:02d}:{secs:02d}", end='\r')  # \r to overwrite the line
        time.sleep(1)
        seconds -= 1
    print("Time's up!")

# Example: 10 second countdown
countdown_timer(10)

In [ ]:
import cv2
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import mediapipe as mp
import pandas as pd
import numpy as np
import pyautogui

In [158]:

data = []

hand_module = mp.solutions.hands
hand = hand_module.Hands(
    static_image_mode = False,
    max_num_hands = 2
)

mp_draw = mp.solutions.drawing_utils
vid = cv2.VideoCapture(0)
which_move = 'a'
while True:
    check , img = vid.read()
    img = cv2.flip(img , 1)
    if not check:
        break
    img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img , cv2.COLOR_RGB2GRAY)

    results = hand.process(img)
    if results.multi_hand_landmarks:

        first_hand = results.multi_hand_landmarks[0]

        features = []
        
        for i in first_hand.landmark:
            features.append(round(i.x , 6))
            features.append(round(i.y ,6))

        if which_move=='c':
            features.append(1)
        elif which_move=='m':
            features.append(2)
        else:
            features.append(3)

        data.append(features)

        for i in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(img , i ,hand_module.HAND_CONNECTIONS)
    cv2.imshow('IMAGE' , img)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
vid.release()
cv2.destroyAllWindows()
data = np.array(data)
data_to_save = pd.DataFrame(data)
data_to_save.to_csv('data.csv' , mode = 'a' , header= False , index = False)




In [159]:
model = pd.read_csv('data.csv')
coords = [f'x{i},y{i}' for i in range(1, 22)]
cols = [val for pair in coords for val in pair.split(',')] + ['label']
model.columns = cols
model.to_csv('new_data.csv' , index = False , header = True)



In [160]:
data = pd.read_csv('new_data.csv')
data = data.sample(frac=1).reset_index(drop=True)
y = data['label']
data = data.drop('label' , axis= 1)
x = np.array(data)
x_test , x_train , y_test , y_train = train_test_split(x , y , train_size=0.3)
y_test -=1
y_train -=1
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)



(829, 42)
(829,)
(354, 42)
(354,)


In [161]:
m1 = keras.Sequential([
    layers.Dense(43 , activation='relu'),
    layers.Dense(10 , activation= 'relu'),
    layers.Dense(3 , activation= 'softmax')
])

m1.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)
m1.fit(
    x_train,
    y_train,
    epochs = 50
)

Epoch 1/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2983 - loss: 1.1826
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4032 - loss: 1.0708
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3838 - loss: 1.0624
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5636 - loss: 1.0362
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6107 - loss: 1.0160
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7661 - loss: 0.9926
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6391 - loss: 0.9539
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8330 - loss: 0.9123 
Epoch 9/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8333 - loss: 0.8580
Epoch 10/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8788 - loss: 0.8013
Epoch 11/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7828 - loss: 0.7503
Epoch 12/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9067 - l

In [151]:
test_loss, test_acc = m1.evaluate(x_test, y_test)
print("Test accuracy:", test_acc)
print("Test loss:", test_loss)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8661 - loss: 0.3539  
Test accuracy: 0.8838862776756287
Test loss: 0.33144494891166687


In [164]:
predictions = m1.predict(x_test)

predicted_labels = np.argmax(predictions, axis=1)
print("First 10 predictions:", predicted_labels[:30])
#print("True labels:         ", y_test[:30])

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step  
First 10 predictions: [1 0 1 1 2 0 0 2 1 1 2 0 1 1 1 1 0 2 0 0 0 1 2 1 0 1 0 1 0 0]


In [ ]:
m1.save('mouse.h5')

In [165]:


hand_module = mp.solutions.hands
hand = hand_module.Hands(
    static_image_mode = False,
    max_num_hands = 2
)
prev_x , prev_y  = 0 , 0
smooth_factor = 0.2
mp_draw = mp.solutions.drawing_utils
vid = cv2.VideoCapture(0)
screen_width, screen_height = pyautogui.size()
while 1:
    check , img = vid.read()
    img = cv2.flip(img , 1)
    if not check:
        break
    img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img , cv2.COLOR_RGB2GRAY)

    results = hand.process(img)
    if results.multi_hand_landmarks:
        f = []
        frame = results.multi_hand_landmarks[0]
        for i in frame.landmark:
            f.append(round(i.x , 6))
            f.append(round(i.y , 6))
     
        f = np.array(f)
        
        f= f.reshape(1 , 42)
        pred = m1.predict(f)
        pred = np.argmax(pred)
        print(pred , end = '')


        index_finger = frame.landmark[8]
        x = int(index_finger.x * screen_width)
        y = int(index_finger.y * screen_height)

        smooth_x = int(prev_x * (1 - smooth_factor) + x * smooth_factor)
        smooth_y = int(prev_y * (1 - smooth_factor) + y * smooth_factor)
        if pred == 1:
            pyautogui.moveTo(x, y)
        elif pred == 0:
            pyautogui.click()
            
        for i in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(img , i ,hand_module.HAND_CONNECTIONS)
    cv2.imshow('IMAGE' , img)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━